# FIN411X – Generate Tables and Figures for Replication Package

This notebook:
1. Loads `summary_all_portfolios.csv` (portfolio performance).
2. Loads `chatgpt_survey_raw_id.csv` (ChatGPT survey with `Student_id`).
3. Merges them by student ID.
4. Builds B, C, and D scales (learning & confidence, metacognition, AI literacy).
5. Computes Cronbach's alpha for each scale.
6. Produces all main tables as CSV files:
   - `table1_portfolio_performance.csv`
   - `table2_ai_scales_reliability.csv`
   - `table3_A2_frequency.csv`
   - `table3_A3_frequency.csv`
   - `table3_A5_frequency.csv`
   - `table3_E1_frequency.csv`
   - `table4_correlations_matrix.csv`
   - `table5_ai_scales_vs_performance.csv`
7. Creates and saves figures:
   - `fig1_hist_total_return.png` (histogram of total returns)
   - `fig2_box_scales.png` (boxplots of B, C, D scales)
   - `fig3_scatter_D_vs_return.png` (scatterplot of AI literacy vs total return)

**Instructions**
- Upload `summary_all_portfolios.csv` and `chatgpt_survey_raw_id.csv` in the left-side Files panel.
- Run all cells from top to bottom.


In [ ]:
# Install required library
!pip -q install scipy


In [ ]:
import pandas as pd
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt

print("Libraries imported.")


## 1. Load and merge the datasets

Make sure the two CSV files are uploaded to the Colab environment.


In [ ]:
# Load portfolio performance summary
summary = pd.read_csv("summary_all_portfolios.csv")

# Load ChatGPT survey with Student_id codes (S01–S30)
survey = pd.read_csv("chatgpt_survey_raw_id.csv")

print("Summary shape:", summary.shape)
print("Survey shape :", survey.shape)

display(summary.head())
display(survey.head())

# Clean ID fields just in case there are spaces
summary["student_id"] = summary["student_id"].astype(str).str.strip()
survey["Student_id"]  = survey["Student_id"].astype(str).str.strip()

# Inner join: keep only students in BOTH files
df = summary.merge(survey, left_on="student_id", right_on="Student_id", how="inner")
print("Merged shape:", df.shape)
df.head()


## 2. Identify B, C, and D scale items

We use the survey column positions from the Google Forms export:
- B items (learning & confidence): columns 8–13
- C items (metacognition): columns 14–19
- D items (AI literacy): columns 20–25


In [ ]:
# Inspect indices and names (optional)
print("Survey columns with indices:\n")
for i, col in enumerate(survey.columns):
    print(i, "->", col[:80])

# Select columns for the three Likert scales based on their positions
B_cols = survey.columns[8:14]   # Section B items
C_cols = survey.columns[14:20]  # Section C items
D_cols = survey.columns[20:26]  # Section D items

print("\nB-scale items:\n", B_cols)
print("\nC-scale items:\n", C_cols)
print("\nD-scale items:\n", D_cols)


## 3. Cronbach's alpha function and scale scores


In [ ]:
def cronbach_alpha(items_df: pd.DataFrame) -> float:
    """Compute Cronbach's alpha for a set of Likert items (columns)."""
    items = items_df.dropna()
    k = items.shape[1]
    if k < 2:
        return np.nan
    item_vars = items.var(axis=0, ddof=1)
    total_scores = items.sum(axis=1)
    total_var = total_scores.var(ddof=1)
    alpha = (k / (k - 1.0)) * (1.0 - item_vars.sum() / total_var)
    return float(alpha)

# Build B, C, D scales
df["B_scale"] = df[B_cols].mean(axis=1)
df["C_scale"] = df[C_cols].mean(axis=1)
df["D_scale"] = df[D_cols].mean(axis=1)

alpha_B = cronbach_alpha(df[B_cols])
alpha_C = cronbach_alpha(df[C_cols])
alpha_D = cronbach_alpha(df[D_cols])

print(f"Scale B (learning/conf.) alpha: {alpha_B:.3f}")
print(f"Scale C (metacognition) alpha : {alpha_C:.3f}")
print(f"Scale D (AI literacy) alpha   : {alpha_D:.3f}")


## 4. Table 1 – Portfolio performance descriptives


In [ ]:
# Convert to percentage for nicer reading
df["total_return_pct_100"] = df["total_return_pct"] * 100
df["alpha_annual_pct"] = df["alpha_annual"] * 100

def summary_row(series: pd.Series) -> dict:
    s = series.dropna()
    return {
        "Mean": s.mean(),
        "SD": s.std(ddof=1),
        "Min": s.min(),
        "25%": s.quantile(0.25),
        "Median": s.median(),
        "75%": s.quantile(0.75),
        "Max": s.max(),
    }

table1_specs = {
    "Final portfolio value (AED)": "final_value_aed",
    "Profit / loss (AED)": "pnl_aed",
    "Total return (%)": "total_return_pct_100",
    "Alpha (annual, %)": "alpha_annual_pct",
    "Beta": "beta",
}

rows_t1 = {}
for label, col in table1_specs.items():
    rows_t1[label] = summary_row(df[col])

table1 = pd.DataFrame.from_dict(rows_t1, orient="index")
print("TABLE 1 – Portfolio performance:\n")
display(table1.round(3))

table1.round(3).to_csv("table1_portfolio_performance.csv")
print("Saved: table1_portfolio_performance.csv")


## 5. Table 2 – AI scales (B, C, D): reliability + descriptives


In [ ]:
scales = {
    "B (learning & confidence)": ("B_scale", len(B_cols), alpha_B),
    "C (metacognitive strategies)": ("C_scale", len(C_cols), alpha_C),
    "D (AI literacy / critical awareness)": ("D_scale", len(D_cols), alpha_D),
}

rows_t2 = {}
for label, (col, k, alpha) in scales.items():
    s = df[col].dropna()
    rows_t2[label] = {
        "k_items": k,
        "Cronbach_alpha": alpha,
        "Mean": s.mean(),
        "SD": s.std(ddof=1),
        "Min": s.min(),
        "25%": s.quantile(0.25),
        "Median": s.median(),
        "75%": s.quantile(0.75),
        "Max": s.max(),
    }

table2 = pd.DataFrame.from_dict(rows_t2, orient="index")
print("TABLE 2 – AI scales (B/C/D):\n")
display(table2.round(3))

table2.round(3).to_csv("table2_ai_scales_reliability.csv")
print("Saved: table2_ai_scales_reliability.csv")


## 6. Table 3 – Frequency tables for A2, A3, A5, E1


In [ ]:
def freq_table(series: pd.Series, categories: list) -> pd.DataFrame:
    s = series.dropna()
    counts = s.value_counts().reindex(categories, fill_value=0)
    perc = counts / len(s) * 100
    return pd.DataFrame({"n": counts, "percent": perc})

# Column names for A2, A3, A5, E1
col_A2 = survey.columns[4]
col_A3 = survey.columns[5]
col_A5 = survey.columns[7]
col_E1 = survey.columns[26]

ft_A2 = freq_table(
    survey[col_A2],
    [
        "Never",
        "Less than once per week",
        "About 1–2 times per week",
        "About 3–4 times per week",
        "Almost every day",
    ],
)

ft_A3 = freq_table(
    survey[col_A3],
    [
        "0 prompts (I did not use ChatGPT)",
        "1–3 prompts",
        "4–7 prompts",
        "8–12 prompts",
        "More than 12 prompts",
    ],
)

ft_A5 = freq_table(
    survey[col_A5],
    ["Never", "Rarely", "Sometimes", "Often", "Almost always"],
)

ft_E1 = freq_table(
    survey[col_E1],
    ["Yes, often", "Sometimes", "Rarely", "Never", "I did not use ChatGPT"],
)

print("TABLE 3a – A2 frequency:\n")
display(ft_A2.round(1))
print("TABLE 3b – A3 frequency:\n")
display(ft_A3.round(1))
print("TABLE 3c – A5 frequency:\n")
display(ft_A5.round(1))
print("TABLE 3d – E1 frequency:\n")
display(ft_E1.round(1))

ft_A2.round(1).to_csv("table3_A2_frequency.csv")
ft_A3.round(1).to_csv("table3_A3_frequency.csv")
ft_A5.round(1).to_csv("table3_A5_frequency.csv")
ft_E1.round(1).to_csv("table3_E1_frequency.csv")
print("Saved: table3_A2/A3/A5/E1_frequency.csv files")


## 7. Table 4 – Correlation matrix


In [ ]:
corr_vars = [
    "total_return_pct",
    "alpha_annual",
    "beta",
    "B_scale",
    "C_scale",
    "D_scale",
]

corr_matrix = df[corr_vars].corr()
print("TABLE 4 – Correlation matrix:\n")
display(corr_matrix.round(3))

corr_matrix.round(3).to_csv("table4_correlations_matrix.csv")
print("Saved: table4_correlations_matrix.csv")


## 8. Table 5 – Selected correlations (r, p-values)


In [ ]:
pairs = [
    ("B_scale", "total_return_pct"),
    ("C_scale", "total_return_pct"),
    ("D_scale", "total_return_pct"),
    ("B_scale", "alpha_annual"),
    ("C_scale", "alpha_annual"),
    ("D_scale", "alpha_annual"),
]

rows_t5 = []
for x, y in pairs:
    r, p = stats.pearsonr(df[x], df[y])
    rows_t5.append({"Predictor_scale": x, "Outcome": y, "r": r, "p": p})

table5 = pd.DataFrame(rows_t5)
print("TABLE 5 – AI scales vs performance (r, p):\n")
display(table5)

table5.to_csv("table5_ai_scales_vs_performance.csv", index=False)
print("Saved: table5_ai_scales_vs_performance.csv")


## 9. Figures – Histogram of total returns

Figure 1: Histogram of portfolio total returns (%).


In [ ]:
plt.figure()
plt.hist(df["total_return_pct_100"].dropna(), bins=8, edgecolor='black')
plt.axvline(df["total_return_pct_100"].mean(), linestyle='dashed')
plt.xlabel("Total return (%)")
plt.ylabel("Number of portfolios")
plt.title("Distribution of portfolio total returns (N = {})".format(len(df)))
plt.tight_layout()
plt.savefig("fig1_hist_total_return.png", dpi=300)
plt.show()

print("Saved: fig1_hist_total_return.png")


## 10. Figures – Boxplots of B, C, D scales

Figure 2: Boxplots of learning & confidence (B), metacognitive strategies (C), and AI literacy (D).


In [ ]:
plt.figure()
plt.boxplot(
    [df["B_scale"].dropna(), df["C_scale"].dropna(), df["D_scale"].dropna()],
    labels=["B: learning/conf.", "C: metacog.", "D: AI literacy"],
)
plt.ylabel("Scale score (1–5)")
plt.title("Distributions of ChatGPT-related scales")
plt.tight_layout()
plt.savefig("fig2_box_scales.png", dpi=300)
plt.show()

print("Saved: fig2_box_scales.png")


## 11. Figures – Scatterplot of AI literacy vs total return

Figure 3: Scatterplot of D_scale (AI literacy) vs total return (%).


In [ ]:
plt.figure()
plt.scatter(df["D_scale"], df["total_return_pct_100"])
plt.xlabel("D scale: AI literacy / critical awareness (1–5)")
plt.ylabel("Total return (%)")
plt.title("AI literacy vs portfolio performance")
plt.tight_layout()
plt.savefig("fig3_scatter_D_vs_return.png", dpi=300)
plt.show()

print("Saved: fig3_scatter_D_vs_return.png")
